In [1]:
import geopandas
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import plotly.express as px
import pandas as pd
import numpy as np
import geopandas
from dash.dependencies import Input, Output

/anaconda/envs/jobs/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
data_path = "/home/dgillen/notebooks/Mar21-housing-insecurity/data/"

In [3]:
df = pd.read_csv(data_path + 'processed/hillsborough_fl_processed_expanded_ACS_2017_to_2019_20210225.csv')
ev_cols = ['census_tract_GEOID','eviction-rate-2017','eviction-rate-2018','eviction-rate-2019']
df = df[ev_cols]
df

,census_tract_GEOID,eviction-rate-2017,eviction-rate-2018,eviction-rate-2019
0,12057010103,0.636364,0.636364,0.636364
1,12057011006,0.540000,0.540000,0.540000
2,12057011108,0.400000,0.400000,0.400000
3,12057011203,0.587302,0.587302,0.587302
4,12057011206,0.476923,0.476923,0.476923
...,...,...,...,...
316,12057013204,0.588235,0.588235,0.588235
317,12057013908,0.483516,0.483516,0.483516
318,12057012600,0.571429,0.571429,0.571429
319,12057003000,0.529762,0.529762,0.529762


In [4]:
mt = pd.melt(df, id_vars=['census_tract_GEOID'])
mt[['name1','name2','year']] = mt.variable.str.split('-', expand=True)
mt = mt[['census_tract_GEOID','year','value']]
mt.columns = ['census_tract_GEOID','year','eviction_rate']
mt

,census_tract_GEOID,year,eviction_rate
0,12057010103,2017,0.636364
1,12057011006,2017,0.540000
2,12057011108,2017,0.400000
3,12057011203,2017,0.587302
4,12057011206,2017,0.476923
...,...,...,...
958,12057013204,2019,0.588235
959,12057013908,2019,0.483516
960,12057012600,2019,0.571429
961,12057003000,2019,0.529762


In [5]:
import json
mean_long_fl = -82.640305
mean_lat_fl = 28.027870
with open(data_path + "geo/hillsborough_fl_2010_tracts_formatted.geojson") as json_file:
    ct_fl_geo_json = json.load(json_file)

In [ ]:
fig1 = px.choropleth_mapbox(mt, geojson=ct_fl_geo_json, color='eviction_rate',
                   color_discrete_sequence = px.colors.qualitative.Dark24,
                   animation_frame="year", #animation_group="country",
                   locations="census_tract_GEOID", featureidkey="properties.census_tract_GEOID",opacity=0.5,
                   mapbox_style="streets")
fig1.update_layout(height=1000,title = "Eviction Rate by Census Tract OVER TIME")
fig1.update_layout(mapbox_style="streets", mapbox_accesstoken=os.environ["MAPBOX_ACCESS_TOKEN"],
                        mapbox_zoom=9, 
                        mapbox_center = {"lat": mean_lat_fl, "lon": mean_long_fl}
                        )  
fig1.show()

In [ ]:
fig1.write_html("EvRateByTractOverTime.html")